Calibration: IMG_7849.JPG

Vega: IMG_7850.JPG

Arcturus: IMG_7854.JPG

Dubhe: IMG_7862.JPG


In [ ]:
import numpy as np
from PIL import Image
from PIL import ImageOps, ImageEnhance, ImageFilter
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from astropy.io import fits

In [ ]:
def set_matplotlib_defaults():
	global FIGURE_WIDTH_1COL
	global FIGURE_WIDTH_2COL
	global FIGURE_HEIGHT_1COL_GR
	global FIGURE_HEIGHT_2COL_GR
	FIGURE_WIDTH_1COL = 3.404  # For PRX style, change for according to journal
	FIGURE_WIDTH_2COL = 7.057  # For PRX style, change for according to journal
	FIGURE_HEIGHT_1COL_GR = FIGURE_WIDTH_1COL*2/(1 + np.sqrt(5))
	FIGURE_HEIGHT_2COL_GR = FIGURE_WIDTH_2COL*2/(1 + np.sqrt(5))
	plt.rcdefaults()
	plt.rcParams.update({
		'font.size'        : 8,  # For PRX style, change for according to journal
		'figure.facecolor' : '0.9',
		'figure.titlesize' : 'medium',
		'figure.dpi'       : 300,
		'figure.figsize'   : (2*FIGURE_WIDTH_1COL,2*FIGURE_HEIGHT_1COL_GR),
		'axes.titlesize'   : 'medium',
		'axes.axisbelow'   : True,
		'xtick.direction'  : 'in',
		'xtick.labelsize'  : 'small',
		'ytick.direction'  : 'in',
		'ytick.labelsize'  : 'small',
		'image.interpolation': 'none',
		'figure.facecolor' : 'white'
	})
set_matplotlib_defaults()

In [ ]:
def load_band(filename, rotation, laser_height=3365, band_width=10, exif=True, plus_90=False):
	img = Image.open(filename)
	if exif: img = ImageOps.exif_transpose(img)  # rotate image according to metadata
	if plus_90: img = img.transpose(Image.ROTATE_90)
	img = img.rotate(rotation, expand=True, resample=Image.BICUBIC)
	# #only take green channel to better isolate the laser
	# img = img.getchannel("G")

	bad_luminance = np.array(img.convert("L").filter(ImageFilter.GaussianBlur(radius=5)))
	if laser_height == None:
		laser_height, _ = np.unravel_index(np.argmax(bad_luminance), bad_luminance.shape)
	print(laser_height)
	img = img.crop((0, laser_height - band_width, img.width, laser_height + band_width))
	img_g = np.dot(np.array(img)[...,:3], [1,2,1]) # maybe shitty choice, depends on the specs of the camera
	# img_g = img
	# https://guaix.fis.ucm.es/~jaz/lp-book/docs/DSLR-spectra.html
	# img = img.crop((2300,1300,2600,1700))
	spectrum = np.average(img_g, axis=0)
	return img, spectrum


img_slice, spec = load_band("Redshift_fail/IMG_7838.JPG", -np.rad2deg(np.arctan(96 / 861)), None)
plt.plot(spec)
plt.show()
plt.imshow(img_slice, cmap='gray', interpolation='nearest')

### Failed Experiment to Determine Absolute Intensity

In [ ]:
def rgb_to_xyz(lin_rgb, M):
	H, W, _ = lin_rgb.shape
	rgb = lin_rgb.reshape(-1, 3)
	xyz = rgb.dot(M.T)
	return xyz.reshape(H, W, 3)

# --- USER: fill in your ISO 17321 matrix here ---
M = np.array([
	[0.4124, 0.3576, 0.1805],
	[0.2126, 0.7152, 0.0722],
	[0.0193, 0.1192, 0.9505],
])

def load_band_individually(filename, laser_height, band_width, rotation):
	img = Image.open(filename)
	img = ImageOps.exif_transpose(img)  # rotate image according to metadata
	img = img.rotate(rotation)
	# #only take green channel to better isolate the laser
	# img = img.getchannel("G")
	img = img.crop((0, laser_height - band_width, img.width, laser_height + band_width))
	# img = img.crop((2300,1300,2600,1700))
	img_arr = np.array(img)


	spectrum_rgb = np.average(img_arr, axis=0)

	# https://www.dxomark.com/Cameras/Canon/EOS-6D---Measurements
	for i, color in enumerate(["r", "g", "b"]):
		plt.plot(spectrum_rgb[:, i], f"{color}", label=color)


	M_raw2srgb = np.array([
		[ 2.29, -1.44,  0.15],
		[-0.27,  1.63, -0.36],
		[ 0.05, -0.75,  1.70]
	])  # rows: R_s, G_s, B_s; cols: R', G', B'


	Rp = img_arr[..., 0]
	Gp = img_arr[..., 1]
	Bp = img_arr[..., 2]

	raw_stack = np.stack([Rp, Gp, Bp], axis=0).reshape((3,))       # shape: (3, …)
	# raw_stack = np.swapaxes(img_arr, 2, 1)      # shape: (3, …)
	print(raw_stack.shape)
	srgb_lin = M_raw2srgb @ raw_stack               # shape: (3, …)

	# 2) linear sRGB → CIE XYZ (step 4.4)
	# M_srgb2xyz_D50 = np.array([
	#     [0.4360747, 0.3850649, 0.1430804],
	#     [0.2225045, 0.7168786, 0.0606169],
	#     [0.0139322, 0.0971045, 0.7141733]
	# ])
	# 
	# xyz_D50 = M_srgb2xyz_D50 @ srgb_lin
	# Y = xyz_D50[1]

	print(srgb_lin.shape, srgb_lin)
	# spectrum_good = np.average(Y, axis=0)
	# print("")
	for i, color in enumerate(["r--", "g--", "b--"]):
		plt.plot(srgb_lin[i], f"{color}", label=color)
# plt.plot(spectrum_good, f"k", label="Combined r + 2g + b")
# plt.legend()


load_band_individually("Redshift_fail/IMG_7838.JPG", 1600, 50, -np.rad2deg(np.arctan(96/861)))

# plt.show()
# plt.imshow(slice, cmap='gray', interpolation='nearest')

In [ ]:
def load_and_linearize(path):
	laser_height = 1600
	band_width = 50
	rotation = -np.rad2deg(np.arctan(96/861))
	img = Image.open(path).convert("RGB")
	img = ImageOps.exif_transpose(img)  # rotate image according to metadata
	img = img.rotate(rotation)
	# #only take green channel to better isolate the laser
	# img = img.getchannel("G")
	img = img.crop((0, laser_height - band_width, img.width, laser_height + band_width))
	arr = np.asarray(img).astype(float) / 255.0
	mask = arr <= 0.04045
	lin = np.empty_like(arr)
	lin[mask] = arr[mask] / 12.92
	lin[~mask] = ((arr[~mask] + 0.055) / 1.055) ** 2.4
	return lin

def rgb_to_xyz(lin_rgb, M):
	H, W, _ = lin_rgb.shape
	rgb = lin_rgb.reshape(-1, 3)
	xyz = rgb.dot(M.T)
	return xyz.reshape(H, W, 3)

# --- USER: fill in your ISO 17321 matrix here ---
M_raw2srgb = np.array([
	[ 2.29, -1.44,  0.15],
	[-0.27,  1.63, -0.36],
	[ 0.05, -0.75,  1.70]
])  # rows: R_s, G_s, B_s; cols: R', G', B'


lin = load_and_linearize("Redshift_fail/IMG_7838.JPG")
xyz = rgb_to_xyz(lin, M_raw2srgb.T)
Y = xyz[:,:,1]   # the corrected intensity
Y_av = np.average(Y, axis=0)
# Now you can plot or summarize Y along any direction to get your spectrum.
# plt.imshow(Y, cmap='gray', interpolation='nearest')
plt.plot(Y_av)
plt.show()

### Actual Calibration

In [ ]:
def moving_avg(array, delta):
	return np.array([np.mean(array[max(0, i - delta):min(len(array), i + delta + 1)]) for i in range(len(array))])

In [ ]:
def index_of_nearest(value, array):
	return np.argmin(np.abs(array - value))

In [ ]:
 # img_slice, spec = load_band("Redshift_fail/IMG_7849.JPG", 30, 28.25, exif=True)
# img_slice, spec = load_band("Redshift_fail/IMG_7850.JPG",100,28.25, exif=True)
def pixel_to_wl(x):
	calibration = np.array([
		[2746, 474.87],
		[3120, 585.48], #(weird)
		[3230, 614.306],
		[3327,640.225]
	])
	wl = np.interp(x, calibration[:,0], calibration[:,1])
	# linearly extend ends
	wl[x < calibration[0,0]] = (calibration[1,1] - calibration[0,1])/(calibration[1,0] - calibration[0,0]) * (x[x < calibration[0,0]] - calibration[0,0]) + calibration[0,1]
	wl[x > calibration[-1,0]] = (calibration[-2,1] - calibration[-1,1])/(calibration[-2,0] - calibration[-1,0]) * (x[x > calibration[-1,0]] - calibration[-1,0]) + calibration[-1,1]
	return wl

x = np.arange(2500, 3500)
calibration = np.array([
	[2746, 476.487], # Ar
	[3120, 585.48], # Ne
	[3230, 614.306], # Ne
	[3327,640.225], # Ne
	[3430, 667.728] # Ne
])

wl_err = 2.1/2 # magic calculator from the internet

# plt.plot(calibration[:, 0], calibration[:, 1], '+')
def model(x, x0, f,a):
	# return m * x + q
	d = 0.005 * 1E-3
	m = -10000
	p = 6.57E-6
	return a*d*np.sin(np.arctan(p/f *(x - x0)))


fit, fit_err = curve_fit(model, *calibration.T, [0, -10, 1E8], sigma=0.21, absolute_sigma=True)

plt.plot(x, pixel_to_wl(x), "-")
print(fit, fit_err)


plt.plot(x, model(x, *fit))
plt.plot(*calibration.T, "x")
plt.axhline(658.2851, linestyle="dashed")


## Find Wavelengths for calibration

In [ ]:
img = Image.open("Redshift_fail/IMG_7849.JPG")
img = ImageOps.exif_transpose(img)  # rotate image according to metadata
img = img.rotate(28.25, expand=True, resample=Image.BICUBIC)
laser_height = 3385
band_width = 10

img_slice = img.crop((0, laser_height - band_width, img.width, laser_height + band_width))
img_g = np.dot(np.array(img_slice)[...,:3], [1,2,1]) # maybe shitty choice, depends on the specs of the camera
# https://guaix.fis.ucm.es/~jaz/lp-book/docs/DSLR-spectra.html
spec = np.average(img_g, axis=0)

In [ ]:
fig, ax = plt.subplots(2, gridspec_kw={'height_ratios': [5, 1], 'hspace': 0.1})

ra = slice(2500,3500)
pixel_x = np.arange(len(spec), dtype=int)

calibration = np.array([
	[2746, 476.487], # Ar
	[3120, 585.48], # Ne
	[3230, 614.306], # Ne
	[3327,640.225], # Ne
	[3430, 667.728] # Ne
])


dx = calibration[1,0] - calibration[0,0]
dy = calibration[1,-1] - calibration[0,-1]

m = dy/dx

continuum  = moving_avg(spec, 25)
normalized = spec/continuum
wavelengths= m * (np.arange(len(spec))- calibration[0,0]) + calibration[0,1]
wy = np.stack([wavelengths,spec,continuum,normalized,pixel_x], axis=0)[:, ra] #[:, 2400:3500]


ax[0].plot(wy[4], wy[3], label="normalized spectrum")
ax[0].margins(0,0.1)


img_slice_2 = ImageEnhance.Brightness(img_slice).enhance(3)
img_slice_2 = np.array(ImageEnhance.Contrast(img_slice).enhance(3))[:, ra, :]
ax[1].imshow(img_slice_2, cmap='gray', interpolation='nearest')
ax[1].set_axis_off()

for x, c in zip(calibration, ["r", "g", "k", "m", "y"]):
	ax[0].plot([3400], [4], color=c)
	ax[0].axvline(x[0], label=f"{x[1]:.3f} nm ({'Ne' if c != 'r' else 'Ar'}): x = {x[0]:.0f}",color=c,linewidth=0.5)


print(np.argmax(3300 + spec[3300:3400]))
ax[0].legend()
plt.savefig("plots/calibration.pdf", bbox_inches='tight')

## Analyse Stellar Spectra

In [ ]:
def peak_wl_in_range(min,max, wls, spec):
	min_index = index_of_nearest(wls, min)
	index = min_index + np.argmin(spec[min_index:index_of_nearest(wls, max)])
	return wls[index], index

In [ ]:
def normalize_and_plot_spectrum(path, laser_displacement=0, band_width=10, rotation=28.25, plus_90=False, bounds=(385,675), enhance=1, hide_image=False):
	img_slice, spec = load_band(path, rotation, laser_height=3385 + laser_displacement, band_width=band_width, exif=True, plus_90=plus_90)
	fig, ax = plt.subplots(2, gridspec_kw={'height_ratios': [5, 1], 'hspace': 0.15})
	pixel_x = np.arange(len(spec), dtype=int)

	continuum  = moving_avg(spec, 25)
	normalized = spec/continuum
	wavelengths= pixel_to_wl(pixel_x)
	ra = slice(index_of_nearest(bounds[0], wavelengths), index_of_nearest(bounds[1], wavelengths))
	wy = np.stack([wavelengths,spec,continuum,normalized,pixel_x], axis=0)[:, ra] #[:, 2400:3500]

	ax[0].plot(wy[0], wy[3], label="measured, normalized spectrum")
	ax[0].grid()
	ax[0].set_xlabel("Wavelength [nm]")
	ax[0].set_ylabel("Relative Flux")
	ax[0].margins(0,0.1)
	# plt.show()

	if enhance != 1:
		img_slice = ImageEnhance.Brightness(img_slice).enhance(enhance)
		img_slice = ImageEnhance.Contrast(img_slice).enhance(enhance)
	img_slice = np.array(img_slice)[:, ra, :]

	if not hide_image:
		ax[1].set_aspect('auto')
		ax[1].imshow(img_slice, cmap='gray', interpolation='nearest')
		ax[1].set_axis_off()
	else:
		fig.delaxes(ax[1])
	return fig, ax, wy


fig, ax, wy = normalize_and_plot_spectrum("Redshift_fail/IMG_7849.JPG", enhance=3)
# ax[0].axvline(658.2851)

### Redshift

In [ ]:
peak_wl_in_range(660, 680, wy[0], wy[3])

In [ ]:
# fig, ax, wy = normalize_and_plot_spectrum("Redshift_fail/IMG_7850.JPG", laser_displacement=20, bounds=(625,675), hide_image=True)
# fig, ax, wy = normalize_and_plot_spectrum("Redshift_fail/IMG_7854.JPG", laser_displacement=-10, band_width=9, plus_90=True, bounds=(625,675), hide_image=True)
fig, ax, wy = normalize_and_plot_spectrum("Redshift_fail/IMG_7862.JPG", laser_displacement=0, band_width=3, plus_90=True, bounds=(625,675), hide_image=True)

real_Ha = 658.2851
ax[0].axvline(real_Ha, color="green", linestyle="dashed", label=f"Real Hα: {real_Ha:.4f} nm ")
measured_Ha, mHa_index = peak_wl_in_range(real_Ha - 5, real_Ha + 5, wy[0], wy[3])
# mHa_err = 0.5 * (wy[0,mHa_index + 1] - wy[0, mHa_index - 1])/2   # pixel-uncertainty * numerical derivative

ax[0].axvline(measured_Ha, color="orange", label=f"measured Hα ({measured_Ha:.0f} $\pm$ {wl_err:.2}) nm")
ax[0].axvspan(measured_Ha - wl_err, measured_Ha + wl_err, color="orange", alpha=0.18)
redshift_z = (measured_Ha - real_Ha)/real_Ha
ax[0].plot([],[],label=f"Redshift: $z = {redshift_z:.3} \pm {abs(wl_err/real_Ha/redshift_z*100):.0f}\%$")

ax[0].legend()
ax[0].set_title("Dubhe Redshift Calculation")


# Redshift with Hα
# 7850: z = -0.0047
# 7854: z = -0.000239
# 7862: z = 0.000979


# Simbad:
# Vega:  z = -0.000045
# Arcturus: z = - 0.000017
# Dubhe: z = -0.000031
plt.savefig("plots/dubhe_redshift.pdf", bbox_inches='tight')

### Compare with literature spectra

In [ ]:
def get_fits_reference_spectrum(filename, end_resolution=None, wl_bounds=(400,675)):
	hdul = fits.open(filename)
	# header = hdul[1].header
	# hdul.info()
	# print(header)
	data = hdul[1].data
	dtype = [('wavelength', float), ('intensity', float)]
	wl = data.field(0)
	y = data.field(1)
	values = np.array([(wl[i], y[i]) for i in range(len(y))], dtype=dtype)
	# print(values)
	sorted = np.sort(values, order="wavelength")
	wl = np.array([sorted[i][0] for i in range(len(y))])
	y = np.array([sorted[i][1] for i in range(len(y))])

	continuum = moving_avg(y, 7000)
	normalized = y/continuum

	if end_resolution is not None:
		resolution = np.mean(np.abs(np.diff(wl)))
		print(f"{resolution = }")
		resolution_surplus = end_resolution/resolution
		print(f"{resolution_surplus = }")
		resolution_surplus = int(resolution_surplus/2)
		normalized = moving_avg(normalized, resolution_surplus )

	wy = np.stack([wl,y,continuum,normalized,np.arange(len(y))], axis=0)
	if wl_bounds is not None:
		wy = wy[:, index_of_nearest(wl, wl_bounds[0]):index_of_nearest(wl, wl_bounds[1])]

	return wy

# wy_vega = get_fits_reference_spectrum("vega_pb.fts", end_resolution=3)
# wy_arcturus = get_fits_reference_spectrum("arcturus_pb.fts", end_resolution=3)
wy_dubhe = get_fits_reference_spectrum("dubhe_pb.fts", end_resolution=3)
# plt.plot(wy_vega[0], wy_vega[3], linewidth=0.05)
# plt.plot(wy_arcturus[0], wy_arcturus[3], linewidth=0.05)
# plt.plot(wy_dubhe[0], wy_dubhe[3], linewidth=0.05)

In [ ]:
fig, ax, wy = normalize_and_plot_spectrum("Redshift_fail/IMG_7850.JPG", laser_displacement=20, enhance=1.25, bounds=(400,675))
wy_vega = get_fits_reference_spectrum("vega_pb.fts", end_resolution=3)
ax[0].plot(wy_vega[0], 0.75*(wy_vega[3] - 1) + 1, linewidth=1, alpha=0.5, label="rescaled, smoothed, literature spectrum")
ax[0].legend()
ax[0].set_title("Spectrum of Vega")
real_Ha = 658.2851
arrow = dict(facecolor='black', shrink=0.05, width=0.3)
tolerance = 5
voffset = 0.005
noise = np.std(wy[3, index_of_nearest(549, wy[0]):index_of_nearest(575,wy[0])])
print(f"{noise = }")
halpha_x, i_ha = peak_wl_in_range(real_Ha - tolerance, real_Ha + tolerance, wy[0], wy[3])
ax[0].annotate(f"Hα\n$({halpha_x:.0f} \pm {wl_err:.0f})$ nm\nSNR: {((1 - wy[3,i_ha])/noise):.1f}",
			   [halpha_x, wy[3,i_ha]- voffset], xytext=[halpha_x - 7, 0.55], ha="right", arrowprops=arrow)
real_Hb = 486.135
hbeta_x, i_hb = peak_wl_in_range(real_Hb - tolerance, real_Hb + tolerance, wy[0], wy[3])
print(halpha_x, hbeta_x)
ax[0].annotate(f"Hβ\n$({hbeta_x:.0f} \pm {wl_err:.0f})$ nm\nSNR: {((1 - wy[3,i_hb])/noise):.1f}",
			   [hbeta_x, wy[3,i_hb] - voffset], xytext=[hbeta_x + 35, 0.55], ha="left", arrowprops=arrow)
real_Hg = 434.0
hgamma_x, i_hg = peak_wl_in_range(real_Hg - tolerance, real_Hg + tolerance, wy[0], wy[3])
print(halpha_x, hbeta_x)
ax[0].annotate(f"Hγ\n$({hgamma_x:.0f} \pm {wl_err:.0f})$ nm\nSNR: {((1 - wy[3,i_hg])/noise):.1f}",
			   [hgamma_x, wy[3,i_hg]- voffset], xytext=[hgamma_x + 10, 0.7], ha="left", arrowprops=arrow)
real_Hd = 401.2
hdelta_x, i_hd = peak_wl_in_range(real_Hd - tolerance, real_Hd + tolerance, wy[0], wy[3])
ax[0].annotate(f"Hδ\n$({hdelta_x:.0f} \pm {wl_err:.0f})$ nm\nSNR: {((1 - wy[3,i_hd])/noise):.1f}",
			   [hdelta_x, wy[3,i_hd]- voffset], xytext=[hdelta_x + 10, 0.55], ha="left", arrowprops=arrow)
real_He = 584.33436	
he_x, i_hd = peak_wl_in_range(real_He - tolerance, real_He + tolerance, wy[0], wy[3])
ax[0].annotate(f"He-I\n$({he_x:.0f} \pm {wl_err:.0f})$ nm\nSNR: {((1 - wy[3,i_hd])/noise):.1f}",
			   [he_x, wy[3,i_hd]- voffset], xytext=[he_x, 0.55], ha="center", arrowprops=arrow)

plt.savefig("plots/vega_spectrum.pdf", bbox_inches='tight')
print(peak_wl_in_range(510 - 20, 510 + 20, wy[0], wy[3]))

In [ ]:
fig, ax, wy = normalize_and_plot_spectrum("Redshift_fail/IMG_7854.JPG", laser_displacement=-10, band_width=9, plus_90=True, enhance=1.25, bounds=(400,675))
wy_arcturus = get_fits_reference_spectrum("arcturus_pb.fts", end_resolution=3)
ax[0].plot(wy_arcturus[0], 0.75 * (wy_arcturus[3] - 1) + 1, linewidth=1, alpha=0.5, label="rescaled, smoothed, literature spectrum")
ax[0].legend()
ax[0].set_title("Spectrum of Arcturus")
real_Ha = 658.2851
arrow = dict(facecolor='black', shrink=0.05, width=0.3)
tolerance = 5
voffset = 0.005
noise = np.std(wy[3, index_of_nearest(600, wy[0]):index_of_nearest(650,wy[0])])
print(f"{noise = }")
halpha_x, i_ha = peak_wl_in_range(real_Ha - tolerance, real_Ha + tolerance, wy[0], wy[3])
ax[0].annotate(f"Hα\n$({halpha_x:.0f} \pm {wl_err:.0f})$ nm\nSNR: {((1 - wy[3,i_ha])/noise):.1f}",
			   [halpha_x, wy[3,i_ha]- voffset], xytext=[halpha_x - 10, 0.75], ha="right", arrowprops=arrow)
real_Hb = 486.135
hbeta_x, i_hb = peak_wl_in_range(real_Hb - tolerance, real_Hb + tolerance, wy[0], wy[3])
print(halpha_x, hbeta_x)
ax[0].annotate(f"Hβ\n$({hbeta_x:.0f} \pm {wl_err:.0f})$ nm\nSNR: {((1 - wy[3,i_hb])/noise):.1f}",
			   [hbeta_x, wy[3,i_hb] - voffset], xytext=[hbeta_x + 15, 0.75], ha="left", arrowprops=arrow)
real_Hg = 434.0
hgamma_x, i_hg = peak_wl_in_range(real_Hg - tolerance, real_Hg + tolerance, wy[0], wy[3])
print(halpha_x, hbeta_x)
ax[0].annotate(f"Hγ\n$({hgamma_x:.0f} \pm {wl_err:.0f})$ nm\nSNR: {((1 - wy[3,i_hg])/noise):.1f}",
			   [hgamma_x, wy[3,i_hg]- voffset], xytext=[hgamma_x + 15, 0.75], ha="left", arrowprops=arrow)
tolerance = 20

real_He = 584.33436	
he_x, i_hd = peak_wl_in_range(real_He - tolerance, real_He + tolerance, wy[0], wy[3])
ax[0].annotate(f"He-I\n$({he_x:.0f} \pm {wl_err:.0f})$ nm\nSNR: {((1 - wy[3,i_hd])/noise):.1f}",
			   [he_x, wy[3,i_hd]- voffset], xytext=[he_x, 0.7], ha="center", arrowprops=arrow)

real_Hd = 401.2
hdelta_x, i_hd = peak_wl_in_range(real_Hd - tolerance, real_Hd + tolerance, wy[0], wy[3])
ax[0].annotate(f"Hδ\n$({hdelta_x:.0f} \pm {wl_err:.0f})$ nm\nSNR: {((1 - wy[3,i_hd])/noise):.1f}",
			   [hdelta_x, wy[3,i_hd]- voffset], xytext=[hdelta_x + 10, 0.7], ha="left", arrowprops=arrow)
ax[0].margins(0,0.15)
plt.savefig("plots/arcturus_spectrum.pdf", bbox_inches='tight')

In [ ]:
fig, ax, wy = normalize_and_plot_spectrum("Redshift_fail/IMG_7862.JPG", laser_displacement=0, band_width=10, plus_90=True, enhance=0.8, bounds=(400,675))
wy_dubhe = get_fits_reference_spectrum("dubhe_pb.fts", end_resolution=3)
ax[0].plot(wy_dubhe[0], 0.75 * (wy_dubhe[3] - 1) + 1, linewidth=1, alpha=0.5, label="rescaled, smoothed, literature spectrum")
ax[0].legend()
ax[0].set_title("Spectrum of Dubhe")
real_Ha = 658.2851
arrow = dict(facecolor='black', shrink=0.05, width=0.3)
tolerance = 5
voffset = 0.005
noise = np.std(wy[3, index_of_nearest(600, wy[0]):index_of_nearest(650,wy[0])])
print(f"{noise = }")
halpha_x, i_ha = peak_wl_in_range(real_Ha - tolerance, real_Ha + tolerance, wy[0], wy[3])
ax[0].annotate(f"Hα\n$({halpha_x:.0f} \pm {wl_err:.0f})$ nm\nSNR: {((1 - wy[3,i_ha])/noise):.1f}",
			   [halpha_x, wy[3,i_ha]- voffset], xytext=[halpha_x - 10, 0.75], ha="right", arrowprops=arrow)
real_Hb = 486.135
hbeta_x, i_hb = peak_wl_in_range(real_Hb - tolerance, real_Hb + tolerance, wy[0], wy[3])
print(halpha_x, hbeta_x)
ax[0].annotate(f"Hβ\n$({hbeta_x:.0f} \pm {wl_err:.0f})$ nm\nSNR: {((1 - wy[3,i_hb])/noise):.1f}",
			   [hbeta_x, wy[3,i_hb] - voffset], xytext=[hbeta_x + 15, 0.75], ha="left", arrowprops=arrow)
real_Hg = 434.0
hgamma_x, i_hg = peak_wl_in_range(real_Hg - tolerance, real_Hg + tolerance, wy[0], wy[3])
print(halpha_x, hbeta_x)
ax[0].annotate(f"Hγ\n$({hgamma_x:.0f} \pm {wl_err:.0f})$ nm\nSNR: {((1 - wy[3,i_hg])/noise):.1f}",
			   [hgamma_x, wy[3,i_hg]- voffset], xytext=[hgamma_x + 15, 0.75], ha="left", arrowprops=arrow)
tolerance = 20

real_He = 584.33436	
he_x, i_hd = peak_wl_in_range(real_He - tolerance, real_He + tolerance, wy[0], wy[3])
ax[0].annotate(f"He-I\n$({he_x:.0f} \pm {wl_err:.0f})$ nm\nSNR: {((1 - wy[3,i_hd])/noise):.1f}",
			   [he_x, wy[3,i_hd]- voffset], xytext=[he_x, 0.7], ha="center", arrowprops=arrow)

real_Hd = 401.2
hdelta_x, i_hd = peak_wl_in_range(real_Hd - tolerance, real_Hd + tolerance, wy[0], wy[3])
ax[0].annotate(f"Hδ\n$({hdelta_x:.0f} \pm {wl_err:.0f})$ nm\nSNR: {((1 - wy[3,i_hd])/noise):.1f}",
			   [hdelta_x, wy[3,i_hd]- voffset], xytext=[hdelta_x + 10, 0.7], ha="left", arrowprops=arrow)
ax[0].margins(0,0.25)

plt.savefig("plots/dubhe_spectrum.pdf", bbox_inches='tight')